In [3]:
import numpy as np

In [45]:
INPUT_SIZE = 13
HIDDEN_UNITS = [128, 64]
EPOCHS = 1000
LEARNING_RATE = 0.0001

In [37]:
def load_data(npz_path):
    data = np.load(npz_path)
    X, y = data['x'], data['y']
    # Normalize features
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    if y.ndim == 1:
        y = y.reshape(-1, 1)
    return X, y


In [1]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)


In [33]:
class Layer:
    def __init__(self, input_size, output_size):
        # He initialization for ReLU
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        self.bias = np.zeros((1, output_size))
        
    def forward(self, X):
        self.X = X
        self.z = np.dot(X, self.weights) + self.bias
        self.a = relu(self.z)
        return self.a
    
    def backward(self, grad, learning_rate):
        delta = grad * relu_derivative(self.z)
        dW = np.dot(self.X.T, delta)
        db = np.sum(delta, axis=0, keepdims=True)
        
        # Clip gradients to prevent explosion
        dW = np.clip(dW, -1, 1)
        db = np.clip(db, -1, 1)
        
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * db
        grad_prev = np.dot(delta, self.weights.T)
        return grad_prev

In [34]:
class OutputLayer(Layer):
    def forward(self, X):
        self.X = X
        self.z = np.dot(X, self.weights) + self.bias
        return self.z 
    
    def backward(self, grad, learning_rate):
        dW = np.dot(self.X.T, grad)
        db = np.sum(grad, axis=0, keepdims=True)
        
        dW = np.clip(dW, -1, 1)
        db = np.clip(db, -1, 1)
        
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * db
        return np.dot(grad, self.weights.T)

In [36]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size=1):
        self.layers = []
        sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)):
            self.layers.append(Layer(sizes[i], sizes[i+1]))
        self.layers.append(OutputLayer(hidden_sizes[-1], output_size))
    
    def forward(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self, X, y, learning_rate):
        y_pred = self.forward(X)
        m = X.shape[0]
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        grad = (y_pred - y) / m
        for layer in reversed(self.layers):
            grad = layer.backward(grad, learning_rate)

In [42]:
def train_model( input_size, hidden_units, epochs=1000, lr=0.001):
    try:
        X_train, y_train = load_data("train.npz")
        X_val, y_val = load_data("val.npz")
        model = NeuralNetwork(input_size=input_size, hidden_sizes=hidden_units, output_size=1)
        for epoch in range(epochs):
            y_pred = model.forward(X_train)
            loss = np.mean((y_pred - y_train) ** 2)
            model.backward(X_train, y_train, lr)
            val_pred = model.forward(X_val)
            val_loss = np.mean((val_pred - y_val) ** 2)
            if epoch % 100 == 0:
                print(f"Epoch {epoch} | Train Loss: {loss:.4f} | Val Loss: {val_loss:.4f}")
        return model
    except Exception as e:
        print(f"Training failed: {e}")
        return None

In [43]:
def evaluate_model(model):
    if model is None:
        print("Model is None. Training failed.")
        return
    X_test, y_test = load_data("test.npz")
    y_pred = model.forward(X_test)
    mae = np.mean(np.abs(y_pred - y_test))
    mse = np.mean((y_pred - y_test) ** 2)
    print(f"\nTest Results:\nMSE: {mse:.4f}\nMAE: {mae:.4f}")

In [46]:
model = train_model(
                       input_size=INPUT_SIZE,
                       hidden_units=HIDDEN_UNITS,
                       epochs=EPOCHS,
                       lr=LEARNING_RATE)
    
evaluate_model(model)

Epoch 0 | Train Loss: 629.9011 | Val Loss: 626.6075
Epoch 100 | Train Loss: 504.2722 | Val Loss: 505.6309
Epoch 200 | Train Loss: 386.2849 | Val Loss: 389.7863
Epoch 300 | Train Loss: 277.7055 | Val Loss: 283.9707
Epoch 400 | Train Loss: 183.5443 | Val Loss: 192.2703
Epoch 500 | Train Loss: 113.3449 | Val Loss: 124.4426
Epoch 600 | Train Loss: 74.0314 | Val Loss: 86.2232
Epoch 700 | Train Loss: 56.9585 | Val Loss: 69.4695
Epoch 800 | Train Loss: 47.2022 | Val Loss: 60.0800
Epoch 900 | Train Loss: 40.0582 | Val Loss: 53.2258

Test Results:
MSE: 38.8270
MAE: 5.1821
